In [2]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
using JLD

phalp_bow_esm = load("../data/phalp_bow_esm.jld")["embedded_bow_esm"]
phalp_bow_rand = load("../data/phalp_bow_rand.jld")["embedded_rand_bow"]
phalp_cnn_esm = load("../data/phalp_CNN_esm.jld")["embedded_CNN_esm"]
upi2doms = load("../data/phalp_upi2doms.jld")["upi2doms"]
domacc2domname = load("../data/phalp_domacc2domname.jld")["domacc2domname"]
phalp_cnn_rand = load("../data/phalp_cnn_rand.jld")["embedded_rand_CNN"]
non_ml = load("../data/phalp_non_ml.jld")["up_evd_type"]
ml_pred = load("../data/phalp_ml.jld")["up_evd_type_ML"]
up2type = load("../data/phalp_type.jld")["up2type"]


┌ Warning: type InlineStrings.String15 not present in workspace; reconstructing
└ @ JLD /home/mfat/.julia/packages/JLD/S6t6A/src/jld_types.jl:697
┌ Warning: type InlineStrings.String31 not present in workspace; reconstructing
└ @ JLD /home/mfat/.julia/packages/JLD/S6t6A/src/jld_types.jl:697


┌ Warning: type InlineStrings.String31 not present in workspace; reconstructing
└ @ JLD /home/mfat/.julia/packages/JLD/S6t6A/src/jld_types.jl:697


Dict{String, Union{Missing, String}} with 11549 entries:
  "UPI000178DD30" => "endolysin"
  "UPI0013EFDC93" => "endolysin"
  "UPI000172D062" => "VAL"
  "UPI001463ABBB" => "endolysin"
  "UPI000232F56D" => "endolysin"
  "UPI0011625D30" => "VAL"
  "UPI0009882324" => "endolysin"
  "UPI000CA1D611" => "VAL"
  "UPI0006BC2F8A" => "endolysin"
  "UPI000BBF7878" => "endolysin"
  "UPI00138B2696" => "endolysin"
  "UPI00025D6AED" => "endolysin"
  "UPI0010C2D3EE" => "endolysin"
  "UPI000D22144F" => "VAL"
  "UPI0018623B24" => "endolysin"
  "UPI00080F0655" => "endolysin"
  "UPI00022BD3A3" => "endolysin"
  "UPI0008093543" => "endolysin"
  "UPI001463E938" => "VAL"
  ⋮               => ⋮

In [3]:
new = filter(x -> (first(x) in collect(keys(non_ml))), up2type)

Dict{String, Union{Missing, String}} with 4829 entries:
  "UPI000178DD30" => "endolysin"
  "UPI0012B4B15F" => "endolysin"
  "UPI0013EFDC93" => "endolysin"
  "UPI000201BE9F" => "endolysin"
  "UPI000172D062" => "VAL"
  "UPI000232F56D" => "endolysin"
  "UPI0011625D30" => "VAL"
  "UPI000CA1D611" => "VAL"
  "UPI000DF0A1E1" => "endolysin"
  "UPI00138B2696" => "endolysin"
  "UPI000012EA4B" => "endolysin"
  "UPI000D22144F" => "VAL"
  "UPI001860540C" => "VAL"
  "UPI001AF87A26" => "VAL"
  "UPI000F4325E4" => "VAL"
  "UPI0012AA2260" => "VAL"
  "UPI000172D02D" => "VAL"
  "UPI0005362DA9" => "VAL"
  "UPI0004593892" => "VAL"
  ⋮               => ⋮

In [4]:
val = filter(x -> (last(x) == "VAL"), new)
endo = filter(x -> (last(x) == "endolysin"), new)

Dict{String, Union{Missing, String}} with 2803 entries:
  "UPI000178DD30" => "endolysin"
  "UPI0012B4B15F" => "endolysin"
  "UPI0013EFDC93" => "endolysin"
  "UPI000201BE9F" => "endolysin"
  "UPI000232F56D" => "endolysin"
  "UPI000DF0A1E1" => "endolysin"
  "UPI00138B2696" => "endolysin"
  "UPI000012EA4B" => "endolysin"
  "UPI000178C353" => "endolysin"
  "UPI001436E76F" => "endolysin"
  "UPI000F6BA7D8" => "endolysin"
  "UPI00001A38F6" => "endolysin"
  "UPI00001A9BAB" => "endolysin"
  "UPI001435C02C" => "endolysin"
  "UPI0010B96635" => "endolysin"
  "UPI0015F22725" => "endolysin"
  "UPI000D7DCFA8" => "endolysin"
  "UPI0010B8D78B" => "endolysin"
  "UPI000EB70E47" => "endolysin"
  ⋮               => ⋮

# Type classifier

In [5]:
function simp_trainer(seq_dict, feature_dict)
"""
input: dict(names, sequences) and dict(names, class)
"""

    inv_fd = Dict()
    for k in keys(feature_dict)
        if feature_dict[k] in keys(inv_fd)
            push!(inv_fd[feature_dict[k]],k)
        else
            inv_fd[feature_dict[k]] = [k]
        end
    end

    types = collect(keys(inv_fd))

    class_hdvs = Dict()
    for i in types
        names_intr = inv_fd[i]
        seq = [seq_dict[n] for n in names_intr]
        class_hdvs[i] = bitadd(hcat(seq)...)
    end

    return class_hdvs
end


simp_trainer (generic function with 1 method)

In [10]:
function simp_train2test(seq, feature)
    """
    input: dict(names, sequences) and dict(names, class)
    """
        class_hdvs = Dict()
        Threads.@threads for i in 1:length(feature)
            if feature[i] in collect(keys(class_hdvs))
                class_hdvs[feature[i]] = bitadd(class_hdvs[feature[i]], seq[i])
            else
                class_hdvs[feature[i]] = seq[i]
            end
        end
    
        return class_hdvs
    end
    

simp_train2test (generic function with 1 method)

In [33]:
using MLDataPattern
function simp_test(seq_dict, feature_dict)
"""
input: dict(names, sequences) and dict(names, class), output simp_trainer
"""
    Keys = [key for (key, val) in feature_dict]
    key_seq = [seq_dict[i] for i in Keys]
    Values = [val for (key, val) in feature_dict]

    (X_train,y_train), (X_test,y_test) = stratifiedobs((key_seq, Values), p = 0.8)
    classes = collect(Set(y_test))
    println(classes)
    n = length(classes)

    class_ind = Dict()
    Threads.@threads for i in 1:n
        class_ind[classes[i]] = i
    end

    confusion_matrix = zeros(n,n)

    class_hdv = simp_train2test(X_train, y_train)

    Threads.@threads for i in 1:length(X_test)
        for j in classes
            simil = 0
            pred = ""
            new_simil = hamming(X_test[i], class_hdv[j])
            if new_simil > simil
                simil = new_simil
                pred = j
            end
        end
        confusion_matrix[class_ind[y_test[i]], class_ind[]] += 1
    end
    return confusion_matrix
end

simp_test (generic function with 1 method)

In [34]:
simp_test(phalp_cnn_rand, new)

["VAL", "endolysin"]


TaskFailedException: TaskFailedException

    nested task error: MethodError: no method matching getindex(::Dict{Any, Any})
    Closest candidates are:
      getindex(::Dict{K, V}, !Matched::Any) where {K, V} at dict.jl:496
      getindex(::AbstractDict, !Matched::Any) at abstractdict.jl:533
      getindex(::AbstractDict, !Matched::Any, !Matched::Any, !Matched::Any...) at abstractdict.jl:543
    Stacktrace:
     [1] macro expansion
       @ ~/Master-Thesis/notebooks/PhaLP_traintest.ipynb:34 [inlined]
     [2] (::var"#278#threadsfor_fun#109"{var"#278#threadsfor_fun#103#110"{Dict{Any, Any}, Matrix{Float64}, Dict{Any, Any}, Vector{String}, SubArray{String, 1, Vector{String}, Tuple{Vector{Int64}}, false}, SubArray{BitVector, 1, Vector{BitVector}, Tuple{Vector{Int64}}, false}, UnitRange{Int64}}})(tid::Int64; onethread::Bool)
       @ Main ./threadingconstructs.jl:84
     [3] #278#threadsfor_fun
       @ ./threadingconstructs.jl:51 [inlined]
     [4] (::Base.Threads.var"#1#2"{var"#278#thread

In [9]:
new["UPI001463ABBB"]

KeyError: KeyError: key "UPI001463ABBB" not found

# Domain classifier